In [1]:
import json
import requests
import pandas as pd
from os.path import join
from os import listdir
from datetime import datetime, timedelta
from renew_case import gen_df_case, CasePair

import re
from req_api import Chart

In [2]:
casepath = join('case', listdir('case')[0])

In [3]:
df = pd.read_csv(casepath, parse_dates=['最後看診日期'], infer_datetime_format=True)
df.rename(columns={'身分證號': 'id', '最後看診日期': 'time'}, inplace=True)
df.drop_duplicates(subset=['id', 'time'], inplace = True)

In [4]:
yrago, case_dict = gen_df_case(df, 'id', 'time')

In [5]:
#case_dict: {(id, visit): [df]...}
case_dict = CasePair(yrago, case_dict)

In [6]:
parsed_list = [{'id_no':k[0], 'visit_date':k[1].strftime('%Y-%m-%d')} for k in case_dict.keys()]

In [7]:
1#attr: df(origin with changed clname, to_datetime(visit_date)), df_json(df: diagnosis to igt and drop previous)
class Old(Chart):
  def __init__(self, parsed_list, case_dict):
    
    super().__init__(parsed_list)
    
#diagnosis: dm == 0, igt == 1, other == 2
    self.df.rename(columns = {'id_no': 'id', 'visit_date': 'time', 'd_case_type':'case'}, inplace = True)
    
    self.df_json = self.df.copy()
    self.df_json['diagnosis'] = self.df_json['diagnosis'].transform(
        lambda x: [int(d['code'][0])%2 for d in x if d['code']=='7902' or re.match('^25', d['code'])])
    self.df_json['diagnosis'] = self.df_json['diagnosis'].transform(lambda v: v[0] if v else 2)
    
#drop previous data    
    self.df_json.drop(index = self.df_json[self.df_json['time']<datetime(2022,7,9)].index, inplace=True)
#drop identity == '自費'
    self.df_json.drop(index = self.df_json[self.df_json['identity']=='自費'].index, inplace=True)
    
    self.df_json.drop_duplicates(subset=['id', 'time', 'case'], inplace = True)

#loc    
    self.df_json = self.df_json[['id','time','case','diagnosis', 'treatments']]
    
    self.create_casedict(case_dict)
#merge ckdonly and 複診, replace中文with代碼, merge新舊and create new case_dict
  def create_casedict(self, case_dict):
        
    copy_v0 = lambda v0: v0.copy() if isinstance(v0, pd.DataFrame) else v0
    
    self.case_dict = {}
        
#因為後面會動到case_dict，用copy比較好
    for k, v in case_dict.items():

      self.case_dict[k] = [copy_v0(v[i]) if i == 0 else v[i] for i in range(len(v))]

#default mark second as true
    duplicated_visit = self.df_json.loc[self.df_json.duplicated(subset=['id', 'time']), ['id','case','time']]
    
#default keep first
    self.df_json.drop_duplicates(subset=['id', 'time'], inplace = True)
    
#merge only ckd with original df
    self.df_json = self.df_json.merge(duplicated_visit, on=['id', 'time'], how = 'outer')
    
#現在只有CKD初診會當日再開(entry和當日其他方案會分開)
    self.df_json['case'] = self.df_json.apply(lambda arrlike, x, y: arrlike[x] if arrlike[y]!='Only CKD初診'
                                              else ('CKD初診+DM複診' if arrlike[x]=='方案 - 複診' else arrlike[y]), 
                                              axis=1, x='case_x', y = 'case_y')
    
    self.df_json.drop(columns=['case_x', 'case_y'], inplace=True)
    
#把格式改成g
    replace_dict = {'方案 - 複診':'is1408', '方案 - 年度':'is1409', 
                 '雙案 複診':'is7001', '雙案 年度':'is7002', 'Only CKD初診': 'is4301CKD',
                 'CKD初診+DM複診':'is4302CKD', '非方案 - 複診':'not1408',
       '非方案 - 年度':'not1409', '非方案 - 一般':'general1408', '方案 - 初診':'is1407', '非方案 - 初診':'not1407'}

    self.df_json.replace({'case': replace_dict}, inplace = True)    

    json_group = self.df_json.groupby('id')
    
#因為key用tuple，所以只能用case_dict當loop主體(不然就要用{id:{t1:[], t2:[]},...}
    for k, v in self.case_dict.items():
#新平台有資料，merge        
      if k[0] in json_group.indices:
            
        newpartv = json_group.get_group(k[0])

        if isinstance(v[0], pd.DataFrame):
                  
          v[0] = v[0][['time','case','igt']].merge(newpartv[['time','case', 'diagnosis']], 
                                                  how = 'outer', on=['time','case'], sort=True)
        
          v[0]['diagnosis'].update(v[0]['igt'])
            
          v[0].drop(columns = 'igt', inplace = True)
            
        else:
          v[0] = newpartv.copy()

          v += [pd.NA, pd.NA]
            
#delete和visit同天的方案
        v[0].drop(index = v[0][v[0]['time']==k[1]].index, inplace =True)
    
#add結案碼
        for i in newpartv.index:
          for d in newpartv.at[i, 'treatments']:
            if d['treatmentHospCode'] == 'P000C':
              v[1] = newpartv.at[i, 'time']
            elif d['treatmentHospCode'] == 'P000D':
              v[2] = newpartv.at[i, 'time']
            
#elif新平台沒有資料只有舊平台有，必須把igt column改成diagnosis
      elif isinstance(v[0], pd.DataFrame):

        v[0]['diagnosis'] = v[0]['igt'].notna()
        v[0].drop(columns = 'igt', inplace = True)

  def predict(self, df):

    def _is(g):

#for即將超過一年
      a_year_minus=timedelta(days=275)

#if DM快過期，提醒當日申報
      last_dm_visit = g.at[g[g.case!='is4301CKD'].index[0],'time']
      if visit - last_dm_visit > a_year_minus:
#if回診會過期=>非方複
        if visit - last_dm_visit > a_year:
          df.loc[df.id == name, 'predict']= '非方複'
          return
 
        df.loc[df.id == name, 'predict']+= 'DM快過期'
    
#if快過期，提醒當日申報CKD(以後可刪，因為會和dm併一起)      
      ckd_g = g[g.case.str.contains(r'CKD|^is7')]
      if not ckd_g.empty:
        last_ckd_visit = g.at[ckd_g.index[0],'time']
        if visit - last_ckd_visit > a_year_minus and visit - last_ckd_visit <= a_year:            
          df.loc[df.id == name, 'predict']+= 'CKD快過期'

      anchor = False
#count只計複診
      count = 0
      mid_case = {'is1408','is7001'}
      dm_anchor = {'is1407', 'is1409', 'is1409new', 'is1409old'}
      ckd_anchor = {'is1407new', 'is4301CKD', 'is4302CKD', 'is7002'}
    
#從最近開始往回找錨點，if錨點在今年，要再往前計入今年所有複診
      for i in g.index:

        if g.at[i, 'case'] in mid_case:
#如果已經有錨點=>確認是否今年=>要計入count
          if anchor:
            if g.at[i, 'time'].year == visit.year:
#只有預測複診會進入，if count>=3 =>非方複
              count += 1
              if count >=3:
                df.loc[df.id == name, 'predict']= '非方複'
                return

#if回溯到去年, return(之前預測的複診不變)
            else:
              return
        
#anchor false(尚未找到錨點)=>如果count超過直接生年度就好
          else:
            count+=1
#如果數到超過3次複，且entry已經到底
            if count>=3 and i == g.index[-1]:
#其他會繼續數，直到找到anchor或數到底
              df.loc[df.id == name, 'predict']+= '?年度'
              return
            
#如果anchor+且複診(年度會當場確認+修正=>return),才會往前計算count(只計1408/7001其他跳過)
#if anchor- and entry belongs to anchor
        elif not anchor and g.at[i, 'case'] in dm_anchor|ckd_anchor:
        
          anchor = g.at[i, 'case']
        
#如果當年度方案已經超過4次，非方
          if count >3:              
            df.loc[df.id == name, 'predict']= '非方複'
            return
#casecount用來predict原始方案，不用來計數當年度方案及當年度複診是否超過        
          casecount = count+1
    
          if anchor in {'is1407', 'is4301CKD', 'is4302CKD'}:
            casecount += 1
#4302/4301上次可能是1407/1409(除非剛好是g.index[-1]會看不到前一個方案)
            if anchor in {'is4301CKD', 'is4302CKD'}:
#if 4302/4301在最後一個entry, 前面可能是1409或沒抓到的1407
              if i == g.index[-1]:
                df.loc[df.id == name, 'predict']= '?'
            
              elif g.at[i+1, 'case'] == 'is1407':
                casecount += 1
                anchor = 'is1407new'

          if anchor in dm_anchor:
            
            if casecount >=4:
              df.loc[df.id == name, 'predict']+= '方案年'
            else:
              df.loc[df.id == name, 'predict']+= '方案複'+str(casecount)
            
          else:

            if casecount >=4:
              df.loc[df.id == name, 'predict']+= '雙案年'
            else:
              df.loc[df.id == name, 'predict']+= '雙案複'+str(casecount)
            
#方案初和方案在去年，不用計數今年度方案 , return           
          if anchor in {'is1407', 'is1407new'} or g.at[i, 'time'].year != visit.year:
            return
        
#只要確定當年年度無重複申報(only happens when predict = p1409 or p7002)，predict年度可return
          if casecount >=4:
            if anchor == 'is1409' or anchor == 'is7002':
              df.loc[df.id == name, 'predict']+= '非方複'
            return
        
#anchor == 7002，4302要改成1408才能計入count複診
          if anchor == 'is7002':
            g['case'] = g['case'].replace('is4302CKD', 'is1408')

    def _not(g):
    
      _g = g.drop(index = g[g.case=='general1408'].index)
      _g.reset_index(drop=True, inplace=True)
    
      ninty_day=timedelta(days=90)
      oneforty_day=timedelta(days=140)
#retro to not1409
      f=_g.case.str.contains(r'1409|1407')
#非年非初起始index為0
      anchoridx = _g[f].index
        
#有anchor    
      if not anchoridx.empty:
            
#if超過4次非複,直接接年     
        if anchoridx[0]>=4:
          df.loc[df.id == name, 'predict']+= '非方年'

#非年=>非複4=>非年
        else:
          df.loc[df.id == name, 'predict']+= '非方複'+str(anchoridx[0]+1)
            
#無anchor, if >=4非方複->非年 
      elif _g.index[-1] >= 3:      
        df.loc[df.id == name, 'predict']+= '非方年'
        
#if最近沒有被結案|距離結案超過365天
      if pd.isna(dm_end) or visit-dm_end>= a_year:

#距離上次回診(包含general)<=90天
        if visit-g.at[g.index[0], 'time']<=ninty_day:

#+距離非方案初or非方案年>=140=>查收案
          if visit-_g.at[_g.index[-1], 'time']>=oneforty_day:
          
            df.loc[df.id == name, 'predict']+= '收方案?'
#距離上次回診(包含general)>90天，且預測非年，提醒是否改方案
        elif _g.index[-1]>=4:
          df.loc[df.id == name, 'predict']+= '無法收方案請人工查是否改為非方複'
        
#predict  
    df['predict']=''
    
#for驗血糖單
    seventy_day=timedelta(days=71)
#for年
    a_year=timedelta(days=365)
    
#dict for change_done
#7002->1409new:雙案年和單案年可各有一次
#可能會ckd結案/dm結案都碰到
#1409old同時是錨點及4302
    ckdtois={r'is(1408CKD|4302CKD|7001)':'is1408', 'is7002':'is1409new',
               'is1409CKD':'is1409old', 'is4301CKD':'not1408'}
    istonot={'is1408':'not1408', r'is140[79](new|old)?':'not1409'}

    for k, _g in self.case_dict.items():
      if isinstance(_g[0], float) or _g[0].empty:
        continue
      name = k[0]
      visit = k[1]

      g = _g[0]
      ckd_end = _g[1]
      dm_end = _g[2]
    
      general_index = g[g['case']=='general1408'].index
    
#if any unhandled case =>continue
      if (~g['case'].isin({'is1408', 'is1409', 'not1408', 'is1407', 'is7001', 'is7002',
                       'not1409', 'general1408', 'not1407', 'is1409CKD', 'is1408CKD', 'is4302CKD', 'is4301CKD'})).any():

        df.loc[df.id == name, 'predict']='兩年內有特殊方案請人工處理'
        continue
                
#if previous diagnosis is 2, continue
      elif g.at[g.index[-1], 'diagnosis'] == 2:
        continue
        
#if previous diagnosis is 1, continue
      elif g.at[g.index[-1], 'diagnosis'] == 1:
        df.loc[df.id == name, 'predict']='非方案 - 一般'
        continue
        
#if drop all general and empty, continue
      elif g.drop(general_index).empty:
        continue
      
#drop all general except the one with g.index[-1]
      g = g.loc[g.index.difference(general_index).union([g.index[-1]])]
    
#將<結案日期通通改成結案後的樣子for方案預測方便
      
#先改CKD結案再改DM結案
      if pd.notna(ckd_end):
      
        g.loc[g['time']<ckd_end,'case'] = g.loc[g['time']<ckd_end,'case'].replace(regex=ckdtois)

      if pd.notna(dm_end):
      
        g.loc[g['time']<dm_end,'case'] = g.loc[g['time']<dm_end,'case'].replace(regex=istonot)
        
#replace 1408ckd/1409ckd with 7001(ckd結案因為都換成dm代碼所以不影響)(會剩下7002, 7001, 1408, 4301, 4302)
      g['case'].replace(['is1408CKD', 'is1409CKD'], 'is7001', inplace = True)

#sort time with descending and then reset index    
      g.sort_values(by='time', ascending=False, inplace=True)
      g.reset_index(drop=True, inplace=True)

#if最後一次A1C 71天內，驗血糖單, continue
      if visit-g.at[g.drop(g[g.case=='general1408'].index).index[0], 'time']<seventy_day:
        df.loc[df.id == name, 'predict']='非方案 - 一般'
        continue
#elif最後一次A1C超過365天, 非方年
      elif visit-g.at[g.drop(g[g.case=='general1408'].index).index[0], 'time']>a_year:
        df.loc[df.id == name, 'predict']='非方案 - 年度'
        continue
    
##去掉距離前次回診超過365天的
      for i in g.index[:-1]:
        if g.at[i, 'time'] - g.at[i+1, 'time'] > a_year:
          g.drop(index = g.index[i+1:], inplace = True)
          break
        
#有is跑方案
      if not g[g.case.str.contains('is')].empty:

        g.drop(g[g.case=='general1408'].index, inplace=True)
#remove all 'not' 
        f=g.case.str.contains(r'not')
        g.drop(g[f].index, inplace=True)
#reset index since 0
        g.reset_index(drop=True, inplace=True)
        
#if最近方案超過365天，自動結案跑not
        if visit-g.at[0, 'time'] > a_year:
          _not(g)
        else:
          _is(g)
#全not跑非方     
      else:
        _not(g)
        
    return df


In [8]:
_old = Old(parsed_list, case_dict)
final = _old.predict(df)

In [9]:
###月末對方案時用
final.merge(_old.df[['id', 'time', 'case_type', 'is_case_type_matched']], on=['id','time'], how='left').replace(
                        {'predict': {r'.*非方複.*':'非方案 - 複診', r'.*非方年.*':'非方案 - 年度',
                        r'.*雙案年.*':'雙案 年度', r'.*雙案複.*':'雙案 複診',
                        r'.*方案年.*':'方案 - 年度', r'.*方案複.*':'方案 - 複診'}}, regex=True).to_csv(
                        'compare.csv', index=False)

In [9]:
###預測方案時用, 之後自動抓(try ai)
final.to_csv(casepath, index=False)